# Facturación de electricidad conforme al PVPC

Simulación de la facturación eléctrica para instalaciones monofásicas de potencia contratada ≤ 10 kW y voltaje < 1kV (instalaciones típicas de particulares)
Admite como entrada consumos totales (tantos como periodos de discriminación horaria en la tarifa seleccionada) o bien consumos horarios. 
También genera consumos horarios en base a mediciones totales, usando los perfiles de consumo del PVPC, y genera el CSV de consumo horario conforme al formato utilizado por las distribuidoras.

## Facturas:

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
from esiosdata import FacturaElec
from esiosdata.facturapvpc import (TIPO_PEAJE_GEN, TIPO_PEAJE_NOC, TIPO_PEAJE_VHC, 
                                   ZONA_IMPUESTOS_PENIN_BALEARES, ZONA_IMPUESTOS_CANARIAS)
from esiosdata.prettyprinting import *


t_0, t_f = '2016-11-01', '2017-01-05'
f1 = FacturaElec(t_0, t_f, tipo_peaje=TIPO_PEAJE_VHC, consumo=[219, 126, 154],
                 zona_impuestos=ZONA_IMPUESTOS_PENIN_BALEARES)
print_ok(f1)

FACTURA ELÉCTRICA:
--------------------------------------------------------------------------------
* Fecha inicio             	01/11/2016
* Fecha final              	05/01/2017
* Peaje de acceso          	2.0DHS (Vehículo eléctrico)
* Potencia contratada      	3.45 kW
* Consumo periodo          	499.00 kWh
* ¿Bono Social?            	No
* Equipo de medida         	1.73 €
* Impuestos                	Península y Baleares (IVA)
* Días facturables         	65
--------------------------------------------------------------------------------

- CÁLCULO DEL TÉRMINO FIJO POR POTENCIA CONTRATADA:
    3.45 kW * 42.043426 €/kW/año * 60 días (2016) / 366 = 23.78 €
    3.45 kW * 41.156426 €/kW/año * 5 días (2017) / 365 = 1.95 €
     -> Término fijo                                                   25.72 €

- CÁLCULO DEL TÉRMINO VARIABLE POR ENERGÍA CONSUMIDA (TARIFA 2.0DHS):
   *Tramo 1, de 01/11/2016 a 31/12/2016:
    Periodo 1: 0.151329 €/kWh                          -> 30.51€(P1)
    - Peaje de 

### Cambios de tarifa:

In [2]:
f1.tipo_peaje = 2
print_info(f1)

f1.tipo_peaje = 'GEN'
print_cyan(f1)

FACTURA ELÉCTRICA:
--------------------------------------------------------------------------------
* Fecha inicio             	01/11/2016
* Fecha final              	05/01/2017
* Peaje de acceso          	2.0DHA (Nocturna)
* Potencia contratada      	3.45 kW
* Consumo periodo          	499.00 kWh
* ¿Bono Social?            	No
* Equipo de medida         	1.73 €
* Impuestos                	Península y Baleares (IVA)
* Días facturables         	65
--------------------------------------------------------------------------------

- CÁLCULO DEL TÉRMINO FIJO POR POTENCIA CONTRATADA:
    3.45 kW * 42.043426 €/kW/año * 60 días (2016) / 366 = 23.78 €
    3.45 kW * 41.156426 €/kW/año * 5 días (2017) / 365 = 1.95 €
     -> Término fijo                                                   25.72 €

- CÁLCULO DEL TÉRMINO VARIABLE POR ENERGÍA CONSUMIDA (TARIFA 2.0DHA):
   *Tramo 1, de 01/11/2016 a 31/12/2016:
    Periodo 1: 0.151056 €/kWh                          -> 27.28€(P1)
    - Peaje de acceso: 18

### Consumo horario:

In [3]:
consumo_horario = f1.consumo_horario
print_ok('Consumo horario [{}]: {} horas, TOTAL={:.2f} kWh\nHead:\n{}\nTail:{}'
         .format(type(consumo_horario), len(consumo_horario), consumo_horario.sum(), 
                 consumo_horario.head(), consumo_horario.tail())) 

Consumo horario [<class 'pandas.core.series.Series'>]: 1560 horas, TOTAL=499.00 kWh
Head:
fecha
2016-11-02 00:00:00+01:00    0.375806
2016-11-02 01:00:00+01:00    0.405489
2016-11-02 02:00:00+01:00    0.388756
2016-11-02 03:00:00+01:00    0.373022
2016-11-02 04:00:00+01:00    0.365404
Name: kWh, dtype: float64
Tail:fecha
2017-01-05 19:00:00+01:00    0.618926
2017-01-05 20:00:00+01:00    0.633574
2017-01-05 21:00:00+01:00    0.620795
2017-01-05 22:00:00+01:00    0.555414
2017-01-05 23:00:00+01:00    0.474396
Name: kWh, dtype: float64


### Factura a partir de consumo horario:

In [4]:
f2 = FacturaElec(consumo=consumo_horario)
f2

FACTURA ELÉCTRICA:
--------------------------------------------------------------------------------
* Fecha inicio             	01/11/2016
* Fecha final              	05/01/2017
* Peaje de acceso          	2.0A (General)
* Potencia contratada      	3.45 kW
* Consumo periodo          	499.00 kWh
* ¿Bono Social?            	No
* Equipo de medida         	1.73 €
* Impuestos                	Península y Baleares (IVA)
* Días facturables         	65
--------------------------------------------------------------------------------

- CÁLCULO DEL TÉRMINO FIJO POR POTENCIA CONTRATADA:
    3.45 kW * 42.043426 €/kW/año * 60 días (2016) / 366 = 23.78 €
    3.45 kW * 41.156426 €/kW/año * 5 días (2017) / 365 = 1.95 €
     -> Término fijo                                                   25.72 €

- CÁLCULO DEL TÉRMINO VARIABLE POR ENERGÍA CONSUMIDA (TARIFA 2.0A):
   *Tramo 1, de 01/11/2016 a 31/12/2016:
    Periodo 1: 0.125623 €/kWh                          -> 57.55€(P1)
    - Peaje de acceso: 458kWh 

### Reparto de la facturación en otros intervalos:

In [5]:
# Reparto en varias facturas:
import datetime as dt

consumo_total = f2.consumo_horario
t0, tf = consumo_total.index[0].date(), consumo_total.index[-1].date()
div_dias = f2.num_dias_factura // 3
# print(t0, tf, div_dias, f2.num_dias_factura)

c1 = consumo_total.loc[:t0 + dt.timedelta(days=div_dias)].iloc[:-1]
c2 = consumo_total.loc[t0 + dt.timedelta(days=div_dias): t0 + dt.timedelta(days=2*div_dias)].iloc[:-1]
c3 = consumo_total.loc[t0 + dt.timedelta(days=2*div_dias):]
assert c1.shape[0] + c2.shape[0] + c3.shape[0] == consumo_total.shape[0]

facturas = [FacturaElec(consumo=c) for c in [c1, c2, c3]]
print_ok(sum([f.coste_total for f in facturas]))
print_ok(f2.coste_total)

114.9
114.89
